# Blog Post 2 - Webscraping for Filmography Recommendations
In the age of Netflix, Amazon Prime, and Disney+, it seems as though the question on the tip of everyone's tongue is "What should I binge next?" Wouldn't it be lovely to find new movie or TV show recommendations based on your preferences? As such, in this post I will outline how we might be able to achieve the same using a webscraper to extract filmography recommendations. 

Specifically, in this post I will be describing how my webscraper extracts the names movies and TV shows that share actors with my favorite film, "Pride & Prejudice (2005)". I will be recommending new movies and TV shows based on the idea is that if "Pride & Prejudice (2005)" has the same actors as another movie or TV show (let this filmography be X), then it is likely that I would also enjoy filmography X. 

### Part 1: The Webscraper
In simple terms, webscraping refers to the act of extracting data from a selected website. The information or data is then exported into a format that is useful (such as csv or text files) for the user. While it is true that data can be scrapped from the web through manual means (e.g., one such rudimentary methos is the simple copy and paste method), it is often a slow and tedious process. Automating the webscrapping process leads to the aggretation of information at much faster rate and also presents information in an easily interpretable manner. 

The first step in the webscraping process is to activate the conda environment with the `scrapy` package loaded start a new project by typing the following lines of code into the command line. Note that I have enclosed the `environment name` and `project name` within square brackets as these are arbitrary (Beware: you should remove the square brackets when you actually type in the environment and project names of your choice). 
```shell
conda activate [environment name]
scrapy startproject [project name]
```

In my case, my environment name is `PIC16B` and I have decided to name this project `IMDB_scraper`. So the code I type in will be as follows:
```shell
conda activate PIC16B
scrapy startproject IMDB_scraper
```

Now that you have created your project, remember to work in your newly created project directory! You can enter this directory by entering the following in your command line. Recall that my project name is "IMDB_scraper". 
```shell
cd IMDB_scraper
```

You might have noticed that your new project directory contains a number of files and/or folders. However, the ones you need to worry about are the `settings.py` file and `spiders` folder. Before writing our scraper we should add the following line of code to our `settings.py` file.
```python
CLOSESPIDER_PAGECOUNT = 20
```
The reason for this is simple: while we are still writing, testing, and debugging out webscraper, we do not want to risk downloading too much data!

Now that we have dealt with the little details, we can create an `imdb_spider.py` file within our `spiders` folder. Within this file we will be writing our Spider class which inherits from `scrapy.Spider` defines how a given website will be scraped. I've decided to call my Spider class `IMDB_spider` which has the following boilerplate code. 

```python
import scrapy

class IMDB_spider(scrapy.Spider):
    
    name = "imdb_spider"
    
    # The following URL is for the Pride & Prejudice (2005)
    # IMDB page. Note that it may be replaced with any other
    # IMDB url
    start_urls = [
        "https://www.imdb.com/title/tt0414387/"
    ]
```

Within the `IMDB_spider` class, I've implemented three different parsing methods: `parse` which parses and then navigates from the main IMDB filmography page to the "Cast & crew" page, `parse_full_credits` which parses the "Full Cast & Crew" page and navigates to each respective actor's page, and `parse_actor_page` which parses each individual cast member's page and yields their respective films and TV shows in a dictionary. The three methods are described in greater detail below. 

The `parse(self, response)` method parses the main IMDB page for "Pride & Prejudice (2005)" (link: https://www.imdb.com/title/tt0414387/) and navigates to the "Cast & crew" page. Note that the url for the "Cast & crew" page has the format `<original_url>fullcredits/`, hence we have the following: 
```python
def parse(self, response):
    """
    Parses the main "Pride & Prejudice (2005)" IMDB page and 
    navigates to the the Cast & crew page
    """
    cast_crew_url = response.url + "fullcredits"
    # Navigating to the cast & crew page using scrapy.Request
    # Once navigated to the above url, the parse_full_credits method
    # will be implemented
    yield scrapy.Request(cast_crew_url, callback=self.parse_full_credits)
```

The `parse_full_credits(self, response)` method parses the Series Cast section (url: https://www.imdb.com/title/tt0414387/fullcredits) and navigates to the individual actor pages. By using the developer tools embedded into chrome, we note that the cast member information is located within the following `html` code: `<table class="cast_list">`. Hence, we can extract the hyperlink information using the `.attrib` function. However, we note that there are three links associated with each cast member - we only want the link corresponding to the image of the cast member. 

The `cast_links` variable in the following parse method stores the url segments following `https://www.imdb.com` and each cast member url has the format `https://www.imdb.com/name/*`, so we need only search for url segments that begin with `/name`. 
We may then loop through the list of url segments, use the `response.urljoin` function to join the two url segments, and then navigate to the respective actor pages by calling the `parse_actor_page` method.

```python
def parse_full_credits(self, response):
    """
    Parses the Series Cast Page for "Pride & Prejudice (2005)" 
    and navigates to each actor's page
    """
    # Extracting links to the cast page
    cast_links = [actor.attrib["href"] for actor in response.css("table.cast_list a")]
    cast_imgs_links = [link for link in set(cast_links) if link.startswith("/name")]
    
    for cast_link in cast_imgs_links:
        url = response.urljoin(cast_link)
        yield scrapy.Request(url, callback=self.parse_actor_page)
```

The `parse_actor_page(self, response)` method parses through the actor page and yields a dictionary with key-value pairs in the following format:
```python
{
    "actor_name" : actor_name,
    "movie_or_TV_name" : movie_or_TV_name
}
```

After navigating to the actor page (e.g., Keira Knightley's IMDB page has the following url: https://www.imdb.com/name/nm0461136/) we note that the actor name is stored withn `span.itemprop`, making it possible to extract the actor name information. We also note that the actor's filmography section is located within `<div class="filmo-category-section b">`. Therefore, we may define the `parse_actor_page` method as follows:

```python
def parse_actor_page(self, response):
    """
    Parses the actor page and stores the shared actor
    information within a dictionary
    """
    # extracting the actor name
    actor_name = response.css("span.itemprop").get().split("<")[-2].split(">")[-1]
    # all filmography information
    all_films = response.css("div.filmo-category-section b").getall()
    
    # looping through all the filmography information and extracting the 
    # filmography names
    for film in all_films:
        movie_or_TV_name = film.split("<")[-3].split(">")[-1]
        
        # Yielding information in a key-value pair dictionary
        yield {
            "actor_name" : actor_name,
            "movie_or_TV_name" : movie_or_TV_name
        }
```

After having defined the Spider class, we are now ready to run the following command to create a .csv file with the actor names and respective filmography
```shell
scrapy crawl imdb_spider -o movies.csv
```
Once we check the .csv file to make sure that our webscraper works correctly, we can navigate to our `settings.py` file and comment out our earlier placed restriction, `CLOSESPIDER_PAGECOUNT = 20`. Now, we re-run the same command, but instead we store all the shared actor information within `results.csv`. 
```shell
scrapy crawl imdb_spider -o results.csv
```

### Part 2: Deducing Filmography Recommendations

Now that we have scraped the required filmography information, let us read in out .csv file. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
films = pd.read_csv('IMDB_scraper/results.csv')
# Checking our dataset
films.head()

,actor_name,movie_or_TV_name
0,Sylvester Morand,Agora
1,Sylvester Morand,The Seeker: The Dark Is Rising
2,Sylvester Morand,Ancient Rome: The Rise and Fall of an Empire
3,Sylvester Morand,The Brief
4,Sylvester Morand,Pride &amp; Prejudice


We would like to determine the number of shared actors, and we can do so by counting the number of times each movie or TV name is repeated within the films dataset. 

In [13]:
# Storing the unique filmography in an array as strings
unique_films = films['movie_or_TV_name'].unique()
unique_films = np.array(unique_films).astype(str)
unique_films # checking the array

array(['Agora', 'The Seeker: The Dark Is Rising',
       'Ancient Rome: The Rise and Fall of an Empire', ...,
       'VM Show Vol. 2', '41 premis Sant Jordi de cinematografia',
       'Mike Leigh on Secrets and Lies'], dtype='<U86')

Notice that we would like to determine the number of shared actors, which we can safely assume, corresponds to the frequency of each movie/TV show in the above films dataset! Therefore, we might use the following function to count the frequency of each film/TV show

In [15]:
# We will now attempt to find the frequency of each filmography
freq = np.zeros(len(unique_films)) 
# all filmography
all_films = np.array(films['movie_or_TV_name']).astype(str)
# looping through the unique filmography
for n, film_name in enumerate(unique_films): 
    for film in all_films: 
        if film == film_name:
            # incrementing the number of shared
            # actors 
            freq[n] += 1

In [16]:
# we will now check the frequency of each movie/TV show
# which will give us the number of shared actors
freq

array([1., 1., 2., ..., 1., 1., 1.])

Now that we have determined the number of shared actors per filmography, let's create a new dataframe

In [17]:
df = pd.DataFrame()
df['movie/TV show'] = unique_films
df['number of shared actors'] = freq
df

,movie/TV show,number of shared actors
0,Agora,1.0
1,The Seeker: The Dark Is Rising,1.0
2,Ancient Rome: The Rise and Fall of an Empire,2.0
3,The Brief,2.0
4,Pride &amp; Prejudice,37.0
...,...,...
2634,54th Golden Globe Awards,1.0
2635,The Smith &amp; Jones Sketchbook,1.0
2636,VM Show Vol. 2,1.0
2637,41 premis Sant Jordi de cinematografia,1.0


Notice that our fourth film is "Pride & Prejudice (2005)". We can safely remove this from the dataset since we are recommending other movies and TV shows to "Pride & Prejudice (2005)" enthusiasts. 

In [20]:
df = df.drop(4)
df

,movie/TV show,number of shared actors
0,Agora,1.0
1,The Seeker: The Dark Is Rising,1.0
2,Ancient Rome: The Rise and Fall of an Empire,2.0
3,The Brief,2.0
5,Fingersmith,1.0
...,...,...
2634,54th Golden Globe Awards,1.0
2635,The Smith &amp; Jones Sketchbook,1.0
2636,VM Show Vol. 2,1.0
2637,41 premis Sant Jordi de cinematografia,1.0


In [ ]:
# Let's sort the movies and TV shows according the number of shared actors (greatest no. to lowest no.)
shared_actors = sorted(zip(freq, unique_films), reverse=True)

# Extracting the film names
filmography = np.array([])
counts = np.array([])
for x, y in shared_actors:
    filmography = np.append(filmography, y)
    counts = np.append(counts, x)

In [22]:
df_sorted = pd.DataFrame()
df_sorted['movie/TV show'] = filmography
df_sorted['number of shared actors'] = counts
df_sorted

,movie/TV show,number of shared actors
0,Pride &amp; Prejudice,37.0
1,The Bill,13.0
2,Pride and Prejudice Revisited,13.0
3,Pride &amp; Prejudice: The Life and Times of J...,13.0
4,Breakfast,13.0
...,...,...
2634,007 Legends,1.0
2635,'NYPD Blue': A Final Tribute,1.0
2636,'Donnie Darko': Production Diary,1.0
2637,'Calendar Girls': The Naked Truth,1.0


In [23]:
# We drop the first row once again since it is Pride & Prejudice
df_sorted = df_sorted.drop(0)
df_sorted

,movie/TV show,number of shared actors
1,The Bill,13.0
2,Pride and Prejudice Revisited,13.0
3,Pride &amp; Prejudice: The Life and Times of J...,13.0
4,Breakfast,13.0
5,Made in Hollywood,11.0
...,...,...
2634,007 Legends,1.0
2635,'NYPD Blue': A Final Tribute,1.0
2636,'Donnie Darko': Production Diary,1.0
2637,'Calendar Girls': The Naked Truth,1.0


Now, let's recommend the first 10 movies in the above data frame!!

In [25]:
recommendations = df_sorted[0:10]
recommendations

,movie/TV show,number of shared actors
1,The Bill,13.0
2,Pride and Prejudice Revisited,13.0
3,Pride &amp; Prejudice: The Life and Times of J...,13.0
4,Breakfast,13.0
5,Made in Hollywood,11.0
6,Entertainment Tonight,11.0
7,Doctor Who,11.0
8,The Graham Norton Show,10.0
9,Pride &amp; Prejudice: The Bennets,10.0
10,Pride &amp; Prejudice: On Set Diaries,10.0


Notice that the scraper was unable to correctly scraper the ampersand, hence we may change the strings a little to improve the readability of our suggestions!

In [29]:
recommendations = recommendations.replace("Pride &amp; Prejudice: The Bennets", "Pride & Prejudice: The Bennets")
recommendations = recommendations.replace("Pride &amp; Prejudice: The Life and Times of Jane Austen", 
                                          "Pride & Prejudice: The Life and Times of Jane Austen")
recommendations = recommendations.replace("Pride &amp; Prejudice: On Set Diaries", "Pride & Prejudice: On Set Diaries")
recommendations

,movie/TV show,number of shared actors
1,The Bill,13.0
2,Pride and Prejudice Revisited,13.0
3,Pride & Prejudice: The Life and Times of Jane ...,13.0
4,Breakfast,13.0
5,Made in Hollywood,11.0
6,Entertainment Tonight,11.0
7,Doctor Who,11.0
8,The Graham Norton Show,10.0
9,Pride & Prejudice: The Bennets,10.0
10,Pride & Prejudice: On Set Diaries,10.0


Ultimately, we have successfully extracted recommendations for "Pride & Prejudice (2005)" enthusiasts via webscraping! 